In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn import linear_model
import csv

def remove_rank(x):
    return x[x[:5].find(')')+2 if x[:5].find(')')>0 else 0:]

def get_schedule(soup):
    
    items = []
    for item in soup.find_all('td'):
        items.append(item.get_text())

    weeks = items[0::10]
    dates = items[1::10]
    times = items[2::10]
    days = items[3::10]
    winners = items[4::10]
    winner_pts = items[5::10]
    location = items[6::10]
    loser = items[7::10]
    loser_pts = items[8::10]
    notes = items[9::10]

    zipped = list(zip(weeks, winners, winner_pts, location, loser, loser_pts))
    schedule_raw = pd.DataFrame(zipped, columns=['week', 'winner', 'winner_pts', 'location', 'loser','loser_pts',])

    schedule_raw['week'] = schedule_raw['week'].astype(str).astype(int)
    schedule_raw['winner_pts'] = pd.to_numeric((schedule_raw['winner_pts'].astype(str)), errors='coerce')
    schedule_raw['loser_pts'] = pd.to_numeric((schedule_raw['loser_pts'].astype(str)), errors='coerce')
    schedule_raw = schedule_raw.dropna() 
    schedule_raw = schedule_raw[((schedule_raw['winner_pts'] + schedule_raw['loser_pts']) > 0)]

    conditions = [schedule_raw['location'] == '@', schedule_raw['location'] == '', schedule_raw['location'] == 'N']
    location_text = ['away', 'home', 'neutral']
    location_adjustment = [3, -3, 0]
    schedule_raw['location'] = np.select(conditions, location_text)
    schedule_raw['location_adjustment'] = np.select(conditions, location_adjustment)

    schedule_raw['winner'] = schedule_raw['winner'].apply(remove_rank)
    schedule_raw['loser'] = schedule_raw['loser'].apply(remove_rank)

    schedule_raw = schedule_raw.drop_duplicates()


    schedule = schedule_raw.drop(['winner_pts','location','loser_pts','location_adjustment'], axis=1)
    return schedule

def get_teams(schedule):
    return pd.concat([schedule['winner'], schedule['loser']], axis = 0, ignore_index=True).unique().tolist()

def n_max(given_list, k, default = 0):
    length = len(given_list)+1
    if k >= length:
        return default
    given_list.sort()
    return given_list[-k]

def n_min(given_list, k, default = 0):
    length = len(given_list)+1
    if k >= length:
        return default
    given_list.sort()
    return given_list[-(length - k)]

In [19]:
url = "https://www.sports-reference.com/cfb/years/2022-schedule.html" 
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

schedule = get_schedule(soup)

In [20]:
schedule

,week,winner,loser
0,1,Florida Atlantic,Charlotte
1,1,Florida State,Duquesne
2,1,Illinois,Wyoming
3,1,Nevada,New Mexico State
4,1,Nevada-Las Vegas,Idaho State
...,...,...,...
311,5,Utah,Arizona State
312,5,Washington,Stanford
313,5,Western Kentucky,Florida International
314,6,Brigham Young,Utah State


In [ ]:
team_df

In [ ]:
get_hank(schedule)

In [ ]:
team_df.to_csv("Hanks_Method.csv")